In [20]:
#!pip install web3

<bound method HexBytes.hex of HexBytes('0xc0dc0a30bd10fc234c1ca465c52720d857dc2dc275b1c886043d32d8ae812ea44e808339a19b35125244649fe91ebf4e3d7475b5ff2faffe31682065eeb482a91b')>


In [48]:
from eth_account import Account
from eth_utils import keccak
import json

def sign_order(private_key, order):
    # Define the EIP-712 domain separator
    domain_separator = {
        'name': 'SignatureVerifier',
        'version': '1.0.0',
        'chainId': 1,  # Ethereum mainnet chain ID
        'verifyingContract': '0x1c91347f2A44538ce62453BEBd9Aa907C662b4bD'  # Address of the contract
    }

    # Convert bytes to a list of integers
    uid_list = list(order['uid'])

    # Define the EIP-712 order structure
    order_structure = [
        {'name': 'tokenIn', 'type': 'address', 'value': order['tokenIn']},
        {'name': 'amountIn', 'type': 'uint256', 'value': str(order['amountIn'])},
        {'name': 'tokenOut', 'type': 'address', 'value': order['tokenOut']},
        {'name': 'amountOut', 'type': 'uint256', 'value': str(order['amountOut'])},
        {'name': 'validTo', 'type': 'uint256', 'value': str(order['validTo'])},
        {'name': 'maker', 'type': 'address', 'value': order['maker']},
        {'name': 'uid', 'type': 'bytes', 'value': uid_list},
    ]

    # Calculate the hash of the domain separator and order
    domain_separator_hash = keccak(json.dumps(domain_separator, sort_keys=True).encode('utf-8')).hex()
    order_hash = keccak(json.dumps({'types': {'EIP712Domain': domain_separator, 'Order': order_structure}, 'primaryType': 'Order', 'domain': domain_separator, 'message': order}, sort_keys=True).encode('utf-8')).hex()
    
    print(order_hash)
    # Sign the order hash with the private key
    signed_order = Account.signHash(order_hash, private_key)

    return signed_order.signature.hex()


# Example usage
private_key = 'c566a08f7be78ba35ce8da6be052917a3a8d1ec22011808a30184a00b727a777'
order = {
    'tokenIn': '0x6778ac35e1c9aca22a8d7d820577212a89544df9',
    'amountIn': 100000000000000000,
    'tokenOut': '0xb4fbf271143f4fbf7b91a5ded31805e42b2208d6',
    'amountOut': 1000000000000000000,
    'validTo': 1747179215,  # Unix timestamp representing the expiration date/time
    'maker': '0x40Eb9614Ba89633B6D0e9fD8177E86D5971CD596',
    'uid': '0x5B38Da6a701c568545dCfcB03FcB875f56beddC4'  # Example UID as bytes
}

signature = sign_order(private_key, order)
print('Signature:', signature)

0142ee3ec957d60517347740f90a68051cdfc69b772417d248416811d2f9ce9e
Signature: 0xb185119cf09826d8d5a43c537074d12fa054d90a3acfbdb21cec9d45b02f2c477e3e3cc143f32f0195db070f8f810d20f10f641d8104f343e4c1e595dd5407ea1b
